In [30]:
%load_ext autoreload
%autoreload 2

import math
from itertools import product
import uproot
import awkward
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import binned_statistic

import matplotlib
matplotlib.rcParams.update({'font.size': 12})
from calo_likelihood import caloLikelihood
from general_functions import *
from loaders import *
from plotter import *
from categorisation import *
from initializers import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
branches = [
    'run',
    'sub',
    'evt',
    
    'backtracked_pdg',
    'backtracked_e',
    'backtracked_purity',
    'backtracked_completeness',
    'backtracked_overlay_purity',
    'backtracked_end_process',
    
    'backtracked_start_x',
    'backtracked_start_y',
    'backtracked_start_z',
        
    'nplanehits_U',
    'nplanehits_V',
    'nplanehits_Y',
    'trk_score',

    'generation',
    'trk_daughters',
    'shr_daughters',

    'trk_sce_start_x',
    'trk_sce_start_y',
    'trk_sce_start_z',

    'trk_sce_end_x',
    'trk_sce_end_y',
    'trk_sce_end_z',
    
    'trk_theta',
    'trk_phi',
     
    'trk_dir_x',
    'trk_dir_y',
    'trk_dir_z',

    'trk_len',
    'trk_distance',
    'longest',
    
    'trk_mcs_muon_mom',
    'trk_range_muon_mom',
    
    'is_hit_montecarlo_u',
    'is_hit_montecarlo_v',
    'is_hit_montecarlo_y',
    
    'dqdx_u',
    'dqdx_v',
    'dqdx_y',

    'dedx_u',
    'dedx_v',
    'dedx_y',

    'rr_u',
    'rr_v',
    'rr_y',

    'pitch_u',
    'pitch_v',
    'pitch_y',
    
    'dir_x_u',
    'dir_x_v',
    'dir_x_y',
    
    'dir_y_u',
    'dir_y_v',
    'dir_y_y',
    
    'dir_z_u',
    'dir_z_v',
    'dir_z_y',
]

In [3]:
today = '20_06_15'
save_folder_base = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/protons_data_check/'
!mkdir -p $save_folder_base

In [4]:
range_v = np.linspace(0.01,40,100)

mpv_dict = {'muon': {0.3: np.array([75.27703858, 10.50417838,  7.54307147,  6.3057034 ,  5.51378723,
          4.98155275,  4.5114527 ,  4.2295646 ,  4.00244235,  3.81458633,
          3.65600097,  3.51991478,  3.37430958,  3.2733431 ,  3.18349973,
          3.10292319,  3.0301619 ,  2.96406232,  2.88940652,  2.8351536 ,
          2.78510907,  2.73877391,  2.69572769,  2.65561335,  2.60913227,
          2.57456196,  2.5420752 ,  2.51148013,  2.48260854,  2.45531228,
          2.42320901,  2.39900011,  2.37599102,  2.35409134,  2.33321982,
          2.31330322,  2.29427533,  2.27164931,  2.25440982,  2.237883  ,
          2.22202444,  2.20679347,  2.19215275,  2.17462984,  2.16119573,
          2.14824974,  2.13576538,  2.12371809,  2.11208509,  2.0980943 ,
          2.08731822,  2.07689274,  2.066801  ,  2.05702721,  2.04755662,
          2.03612375,  2.02728605,  2.01870958,  2.01038305,  2.00229583,
          1.99443788,  1.98492351,  1.97754765,  1.97037219,  1.96338927,
          1.95659142,  1.94997157,  1.9419369 ,  1.93568739,  1.92959514,
          1.92365143,  1.91785099,  1.91218883,  1.90529834,  1.8999302 ,
          1.89468564,  1.88956055,  1.88455102,  1.87965333,  1.87368303,
          1.8690241 ,  1.86446592,  1.86000538,  1.85563952,  1.85136547,
          1.84614786,  1.84207054,  1.83807652,  1.83416341,  1.83032888,
          1.82657072,  1.82288679,  1.81838314,  1.81485883,  1.81140233,
          1.80801184,  1.80468558,  1.80142188,  1.79742772,  1.79429886]),
  1: np.array([81.82722581, 11.41761901,  8.19858824,  6.85335466,  5.99234856,
          5.41363944,  4.90244939,  4.59589647,  4.34888254,  4.14455794,
          3.97205632,  3.82401622,  3.66560629,  3.55575019,  3.45798811,
          3.37030198,  3.29111362,  3.21916891,  3.13790301,  3.07884027,
          3.02435385,  2.9739012 ,  2.92702518,  2.88333759,  2.83271045,
          2.79505219,  2.75965996,  2.72632513,  2.6948648 ,  2.66511787,
          2.63012811,  2.60373927,  2.5786555 ,  2.55477849,  2.5320199 ,
          2.51030007,  2.48954701,  2.46486633,  2.44605882,  2.42802663,
          2.41072148,  2.39409911,  2.37811894,  2.3589902 ,  2.34432287,
          2.33018665,  2.31655273,  2.3033944 ,  2.29068689,  2.27540152,
          2.26362651,  2.25223306,  2.2412028 ,  2.23051858,  2.22016434,
          2.20766274,  2.19799733,  2.18861625,  2.17950724,  2.1706587 ,
          2.16205973,  2.15164637,  2.1435722 ,  2.13571619,  2.12806979,
          2.12062487,  2.11337374,  2.10457129,  2.09772394,  2.09104782,
          2.08453373,  2.07817592,  2.07196896,  2.06441446,  2.05852824,
          2.05277682,  2.04715574,  2.04166073,  2.03628772,  2.0297371 ,
          2.02462459,  2.019622  ,  2.01472595,  2.0099332 ,  2.00524062,
          1.99951124,  1.99503333,  1.99064632,  1.98634759,  1.98213462,
          1.97800499,  1.97395637,  1.96900611,  1.96513169,  1.96133129,
          1.95760292,  1.95394466,  1.95035468,  1.94596051,  1.94251772])},
 'proton': {0.3: np.array([288.90446811,  33.36008254,  23.24846796,  19.0563639 ,
          16.38287677,  14.58912581,  13.0056833 ,  12.05597818,
          11.29026466,  10.65630501,  10.1204673 ,   9.65999827,
           9.16644576,   8.82353574,   8.51784073,   8.24314586,
           7.99459361,   7.76832568,   7.51214932,   7.32551616,
           7.1529668 ,   6.99283416,   6.84371387,   6.70441345,
           6.5425572 ,   6.42183602,   6.30810183,   6.20071396,
           6.09911074,   6.00279741,   5.88918258,   5.80324486,
           5.72134287,   5.64317458,   5.56846854,   5.49698003,
           5.42848777,   5.34678202,   5.2843253 ,   5.22427628,
           5.16648642,   5.11081965,   5.05715104,   4.99270099,
           4.94312169,   4.89519865,   4.8488431 ,   4.80397279,
           4.76051136,   4.70805809,   4.6675144 ,   4.62816603,
           4.58995637,   4.55283247,   4.51674477,   4.47302174,
           4.43909984,   4.40607316,   4.37390372,   4.34255574,
           4.31199547,   4.27485456,   4.24595295,   4.21774187,
           4.19019476,   4.16328649,   4.13699322,   4.10495713,
           4.07996666,   4.05552175,   4.03160323,   4.00819288,
           3.98527331,   3.95728886,   3.93541418,   3.91397922,
           3.89296974,   3.87237216,   3.85217349,   3.82746722,
           3.80812144,   3.78913607,   3.7705003 ,   3.75220378,
           3.73423657,   3.71222617,   3.69496562,   3.67800476,
           3.66133522,   3.64494894,   3.62883818,   3.61299551,
           3.59355823,   3.5782929 ,   3.56327328,   3.548493  ,
           3.53394595,   3.51962621,   3.5020375 ,   3.48820861]),
  1: np.array([314.04359052,  36.26275106,  25.27119192,  20.7142522 ,
          17.80807707,  15.85819458,  14.13691447,  13.10453082,
          12.27215125,  11.58299214,  11.00049475,  10.4999257 ,
           9.96338809,   9.59061008,   9.25828631,   8.95966102,
           8.68945394,   8.44347097,   8.16497141,   7.96207342,
           7.77448529,   7.60039482,   7.43827542,   7.28683081,
           7.11086252,   6.97961472,   6.85596226,   6.73920868,
           6.62874344,   6.52402869,   6.40050218,   6.30706653,
           6.21801804,   6.13302838,   6.05180249,   5.97407434,
           5.89960338,   5.81076475,   5.74285498,   5.67756261,
           5.61472618,   5.55419776,   5.49584155,   5.42576158,
           5.37185089,   5.31974073,   5.26933461,   5.22054313,
           5.17328325,   5.11624515,   5.07215721,   5.02936874,
           4.98781818,   4.947448  ,   4.90820429,   4.86065708,
           4.8237679 ,   4.78785195,   4.75286793,   4.71877695,
           4.68554235,   4.64515083,   4.61371943,   4.58303874,
           4.55307989,   4.52381556,   4.49521983,   4.46037807,
           4.4331987 ,   4.40661244,   4.38059847,   4.35513696,
           4.33020901,   4.29977207,   4.27598011,   4.2526662 ,
           4.22981487,   4.20741136,   4.18544153,   4.15856857,
           4.137526  ,   4.11687527,   4.09660462,   4.0767028 ,
           4.05715902,   4.03321701,   4.01444152,   3.99599185,
           3.97785889,   3.96003391,   3.94250849,   3.92527452,
           3.90413007,   3.88752381,   3.87118469,   3.8551058 ,
           3.83928048,   3.82370229,   3.80456766,   3.78952316])}}

In [23]:
base_folder = '/home/nic/Desktop/MicroBooNE/bnb_nue_analysis/events_check_calorimetry/'
selected_tracks = [
    ('calo_10857_201_10066.root', 10857, 10066, 50, 60),
    ('calo_10846_207_10366.root', 10846, 10366, 0, 100),
    ('calo_16301_241_12067.root', 16301, 12067, 0, 60),
    
]

In [45]:
for filename, run, event, len_min, len_max in selected_tracks:
    ar = uproot.open(base_folder+filename)['nuselection']['CalorimetryAnalyzer'].arrays(branches, namedecode="utf-8")
    initialisation_for_caloriemtry_data_mc(ar)
    compute_pid(ar)
    event_mask = (ar['run']==run) & (ar['evt']==event) & (ar['trk_len'] > len_min) & (ar['trk_len'] < len_max)
    track_pitch = track_dir_pitch(ar['trk_dir_y'][event_mask][0], ar['trk_dir_z'][event_mask][0])
    print(f"Event {event}, run {run}")
    print(f"Pitch U {track_pitch[0]:.2f}, V {track_pitch[1]:.2f}, Y {track_pitch[2]:.2f}")
    print(f"Track dir x {ar['trk_dir_x'][event_mask][0]:.2f}, y {ar['trk_dir_y'][event_mask][0]:.2f}, Z {ar['trk_dir_z'][event_mask][0]:.2f}")
    print()
    for plane, label in zip(['u', 'v', 'y'], ['First Induction Plane', 'Second Induction Plane', 'Collection Plane']):
        dedx = ar[f'dedx_{plane}'][event_mask][0]
        rr = ar[f'rr_{plane}'][event_mask][0]    
        pitch = ar[f'pitch_{plane}'][event_mask][0]

        plt.plot(range_v, mpv_dict['proton'][0.3], color='red', label='Proton MPV @pitch = 0.3 cm')
        plt.plot(range_v, mpv_dict['muon'][0.3], color='orange', label='Muon MPV @pitch = 0.3 cm')

        plt.plot(rr, dedx, 'k.', label='data')

        plt.xlabel('Residual range [cm]')
        plt.ylabel('dE/dx [MeV/cm]')
        plt.title('MicroBooNE Preliminary', loc='right')
        plt.title(f'Event {event}, run {run}\nproton candidate\n{label}', loc='left')
        plt.ylim(0, 25)
        plt.legend()
        plt.tight_layout()
        plt.savefig(save_folder_base + f'run_{run}_ev_{event}_plane_{plane}.png', dpi=250)
        plt.close()
        
    for plane, label in zip(['u', 'v', 'y'], ['First Induction Plane', 'Second Induction Plane', 'Collection Plane']):
        rr = ar[f'rr_{plane}'][event_mask][0]    
        pitch = ar[f'pitch_{plane}'][event_mask][0]
        theta_yx = ar[f'abs_theta_yx_{plane}'][event_mask][0]
        print(f"plane {plane} with mean pitch {pitch.mean():.2f} cm")
        print(f"plane {plane} with mean phi {theta_yx.mean():.2f}")
        print()
        plt.plot(rr, pitch, label=label)

        plt.xlabel('Residual range [cm]')
        plt.ylabel('Pitch [cm]')
        plt.title('MicroBooNE Preliminary', loc='right')
        plt.title(f'Event {event}, run {run}\nproton candidate\n{label}', loc='left')
        plt.ylim(0, 5)
    plt.legend()
    plt.tight_layout()
    plt.savefig(save_folder_base + f'run_{run}_ev_{event}_pitch.png', dpi=250)
    plt.close()

Event 10066, run 10857
Pitch U 0.98, V 0.45, Y 0.31
Track dir x 0.06, y 0.21, Z 0.98

plane u with mean pitch 0.98 cm
plane u with mean phi 1.51

plane v with mean pitch 0.45 cm
plane v with mean phi 1.50

plane y with mean pitch 0.31 cm
plane y with mean phi 1.33

Event 10366, run 10846
Pitch U 0.47, V 0.92, Y 0.31
Track dir x -0.18, y -0.18, Z 0.97

plane u with mean pitch 0.48 cm
plane u with mean phi 1.42

plane v with mean pitch 0.92 cm
plane v with mean phi 1.40

plane y with mean pitch 0.31 cm
plane y with mean phi 0.95

Event 12067, run 16301
Pitch U 0.42, V -11.75, Y 0.43
Track dir x 0.58, y -0.43, Z 0.70

plane u with mean pitch 0.39 cm
plane u with mean phi 0.62

plane v with mean pitch 6.83 cm
plane v with mean phi 1.02

plane y with mean pitch 0.43 cm
plane y with mean phi 0.75

